In [ ]:
import os

import ase.io
import ase.atoms
import scipy.stats
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

plt.style.use('seaborn-v0_8')

In [ ]:
particles = list(ase.io.iread(os.path.join('data', 'S_random_8000.xyz')))


In [ ]:
particle = particles[0]
# help(particle)
pd.DataFrame(
    particle.get_all_distances().round(2) + np.identity(42), 
    index=particle.get_chemical_symbols(),
    columns=particle.get_chemical_symbols()
).head()

In [ ]:
particle_span = []
for particle in particles:
    particle_span.append(np.max(particle.get_all_distances()))
    


In [ ]:
y = pd.read_csv(os.path.join('data', 'S_random_8000.trans'), header=None)[0]

In [ ]:
scipy.stats.pearsonr(y, particle_span)

In [ ]:
dst_series = pd.Series(particle_span)
dst_series.name = "particle span"
mu, std = scipy.stats.norm.fit(dst_series) 
dst_series.plot.hist(bins=15, density=True, edgecolor = "white")

xmin, xmax = plt.xlim()
x = np.linspace(xmin, xmax, 100)
p = scipy.stats.norm.pdf(x, mu, std)

plt.plot(x, p, 'r',  linewidth=2)
plt.xlabel("Particle Span [Å]")
plt.title("Histogram")
plt.grid(axis='x')

In [ ]:
for i, atom in enumerate(particle):
    print(i, atom)

In [ ]:
# 1. siarka - odłegość między nimi
sulfur_idxes = [10, 31]
# 2. benzeny odległość między środkami
benzene1_idxes = [11, 14, 15, 16, 17, 20]
benzene2_idxes = [21, 24, 25, 26, 27, 30]
# 3. kąt skręcenia benzenów - skręcają się tylko na osi wiązania między 21 a 22 (z obrazka)
benzene1_plane_idxes = [14, 15, 16]   # oba przypadki to V-ki zaczynające się od prawego górenego rogu, potem dolny i potem lewy gorny
benzene2_plane_idxes = [25, 26, 27]
# 4.  kąty międy jakimiś parami atomów Au-S-C, czy C-C-C, może wybór genetyczny?
Au_S_C_idxes_1 = [30, 31, 32]
Au_S_C_idxes_2 = [9, 10, 11]
# 5. wyciągnąć dane ze stinga informacyjnego


    
# 1.
particle.get_distance(*sulfur_idxes)

# 2.
benzene1_center = np.mean(particle.positions[benzene1_idxes], axis=0)
benzene2_center = np.mean(particle.positions[benzene2_idxes], axis=0)
np.linalg.norm(benzene1_center - benzene2_center)

# 3.
#   wektory v1, v2 rozpinające płaszyczyznę
benzene1_v1 = particle.positions[benzene1_plane_idxes[0]] - particle.positions[benzene1_plane_idxes[1]]
benzene1_v2 = particle.positions[benzene1_plane_idxes[2]] - particle.positions[benzene1_plane_idxes[1]]

benzene2_v1 = particle.positions[benzene2_plane_idxes[0]] - particle.positions[benzene2_plane_idxes[1]]
benzene2_v2 = particle.positions[benzene2_plane_idxes[2]] - particle.positions[benzene2_plane_idxes[1]]

#   wektor normalny płaszczyzny - np.cross iloczyn wektoryowy
plane1 = np.cross(benzene1_v1, benzene1_v2)
plane2 = np.cross(benzene2_v1, benzene2_v2)

#   kąt między wektorami
def unit_vector(vector):
    """ Returns the unit vector of the vector.  """
    return vector / np.linalg.norm(vector)


def angle_between(v1, v2):
    """ Returns the angle in radians between vectors 'v1' and 'v2'::

            >>> angle_between((1, 0, 0), (0, 1, 0))
            1.5707963267948966
            >>> angle_between((1, 0, 0), (1, 0, 0))
            0.0
            >>> angle_between((1, 0, 0), (-1, 0, 0))
            3.141592653589793
    """
    v1_u = unit_vector(v1)
    v2_u = unit_vector(v2)
    return np.arccos(np.clip(np.dot(v1_u, v2_u), -1.0, 1.0))


angle_between(plane1, plane2)


# 4.
# permutacje wszystkich kątów Au S C
for i in range(3):
    idxes_1 = [Au_S_C_idxes_1[i%3], Au_S_C_idxes_1[(i+1)%3], Au_S_C_idxes_1[(i+2)%3]]
    idxes_2 = [Au_S_C_idxes_2[i%3], Au_S_C_idxes_2[(i+1)%3], Au_S_C_idxes_2[(i+2)%3]]
    print(particle.get_angle(*idxes_1), particle.get_angle(*idxes_2))
    
# 5.


In [ ]:

xyz = particles[0]

In [ ]:
xyz.get_angles([[0,2, 3]])